In [1]:
# data load

from glob import glob

tweet_list = []
label_list = []

for index, file in enumerate(glob('data/3000tweet/raw_texts/**/*.txt', recursive=True)): # labels
    label_list.append(file.split("\\")[1]),
    tweet_list.append((open(file, encoding="windows-1254").read().replace('\n', ' ').strip().lower()))      # tweets

print("3000 tweet  :  ", len(tweet_list))
print("3000 label  :  ", len(label_list))


3000 tweet  :   3000
3000 label  :   3000


In [2]:
print("toplam veri sayısı : ",len(label_list))
value1 = [i for i in label_list if i in '1']
value2 = [i for i in label_list if i in '2']
value3 = [i for i in label_list if i in '3']
print("1.sınıf : ",len(value1))
print("2.sınıf : ",len(value2))
print("3.sınıf : ",len(value3))

toplam veri sayısı :  3000
1.sınıf :  756
2.sınıf :  1287
3.sınıf :  957


In [ ]:
print("3000 tweet  :  ", tweet_list)

In [4]:
# preprocessing

import Stemmer
import re


def splitIntoStem(message):

    
    return [removeNumeric(stripEmoji(singleCharacterRemove(removePunctuation
                                                           (removeHyperlinks
                                                            (removeHashtags
                                                             (removeUsernames
                                                              (stemWord(word)))))))) for word in message.split()]
def stemWord(tweet):

    ################################## for turkish
    #stemmer = Stemmer.Stemmer('turkish')
    #return stemmer.stemWord(tweet).lower()
    
    return tweet.lower()

#remove usernames
def removeUsernames(tweet):
    return re.sub('@[^\s]+', '', tweet)


#remove hashtag
def removeHashtags(tweet):
    return re.sub(r'#[^\s]+', '', tweet)

#remove link
def removeHyperlinks(tweet):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)

#remove numeric character
def removeNumeric(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

#remove punctuation
def removePunctuation(tweet):

    return re.sub(r'[^\w\s]','',tweet)

#remove single character
def singleCharacterRemove(tweet):
    return re.sub(r'(?:^| )\w(?:$| )', ' ', tweet)

#remove emoji
def stripEmoji(text):

    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', text)

In [5]:
tweet_list[0]

'dun turkcelle tepkilerimizden sonra bugün turkcell twittera sponsor olmuş. ne tesadüf değil mi ? :)'

In [6]:
for i,k in enumerate(tweet_list):
    tweet_list[i] = " ".join(splitIntoStem(k)).split()

In [7]:
tweet_list[0]

['dun',
 'turkcelle',
 'tepkilerimizden',
 'sonra',
 'bugün',
 'turkcell',
 'twittera',
 'sponsor',
 'olmuş',
 'ne',
 'tesadüf',
 'değil',
 'mi']

In [8]:
import string

def sumTweetWords(tweet_list):
    new_value = ""
    for i in tweet_list:
        value = " ".join(i) 
        new_value = new_value + " " +value
    print("len(tweet_list) : ",len(new_value.split()))

sumTweetWords(tweet_list)

len(tweet_list) :  32588


In [9]:
# remove stopwords

from nltk.corpus import stopwords


def removeStopWords(tweet_list):
    
    filtered_stopwords = []
    filtered_stopwords_number = []
    
    stop_words = stopwords.words('turkish')

    stop_words.append("bir")
    stop_words.append("iki")
    stop_words.append("üç")
    stop_words.append("dört")
    stop_words.append("beş")
    stop_words.append("altı")
    stop_words.append("yedi")
    stop_words.append("sekiz")
    stop_words.append("dokuz")
    stop_words.append("on")
    
    print("stop_words : ",stop_words)
    
    for i in tweet_list:
        filtered_sentence = [w for w in i if not w in stop_words]
        filtered_stopwords_number.append(filtered_sentence)                      # return list value
        filtered_stopwords.append(" ".join(filtered_sentence))                   # return string value
    
    return filtered_stopwords,filtered_stopwords_number                                       


tweet_list,filtered_stopwords_number = removeStopWords(tweet_list)

stop_words :  ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'bir', 'iki', 'üç', 'dört', 'beş', 'altı', 'yedi', 'sekiz', 'dokuz', 'on']


In [ ]:
tweet_list

In [ ]:
filtered_stopwords_number

In [12]:
sumTweetWords(filtered_stopwords_number)

len(tweet_list) :  28919


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

msg_train, msg_test, label_train, label_test = \
    train_test_split(tweet_list,label_list, test_size=0.2)  # random splitting

print("\n")
print("\n")
print("Message Train :", len(msg_train), "Message Test : ", len(msg_test), "Sum Data : ",
      len(msg_train) + len(msg_test))

labels = ['1','2','3']


## svm classification

svm = Pipeline([('vect', TfidfVectorizer()), ('svm', LinearSVC())])
svm.fit(msg_train, label_train)
y_pred_class = svm.predict(msg_test)

print("\n")
print("\n")
print('SVM Clasification: \n', classification_report(label_test, y_pred_class))  # classification report
print('SVM Confussion matrix: \n', confusion_matrix(label_test, y_pred_class,labels=labels))  # confusion matrix

print("\n")
print("\n")
print("SVM  Standard Accuracy : :  : \n", accuracy_score(label_test, y_pred_class))  # accuracy
print("\n")
print("\n")
print('SVM F1 score:', f1_score(label_test, y_pred_class, average='weighted'))
print("\n")
print("\n")


## Bayes classification

bayes = Pipeline([('vect', TfidfVectorizer()), ('bayes', MultinomialNB())])
bayes.fit(msg_train, label_train)
y_pred_class = bayes.predict(msg_test)

print("\n")
print("\n")
print('Bayes Clasification: \n', classification_report(label_test, y_pred_class))  # classification report
print('Bayes Confussion matrix: \n', confusion_matrix(label_test, y_pred_class,labels=labels))  # confusion matrix

print("\n")
print("\n")
print("Bayes  Standard Accuracy : :  : \n", accuracy_score(label_test, y_pred_class))  # accuracy
print("\n")
print("\n")
print('Bayes F1 score:', f1_score(label_test, y_pred_class, average='weighted'))
print("\n")
print("\n")







Message Train : 2400 Message Test :  600 Sum Data :  3000




SVM Clasification: 
               precision    recall  f1-score   support

           1       0.52      0.50      0.51       155
           2       0.61      0.73      0.67       256
           3       0.46      0.35      0.40       189

    accuracy                           0.55       600
   macro avg       0.53      0.53      0.52       600
weighted avg       0.54      0.55      0.54       600

SVM Confussion matrix: 
 [[ 77  40  38]
 [ 28 188  40]
 [ 44  79  66]]




SVM  Standard Accuracy : :  : 
 0.5516666666666666




SVM F1 score: 0.540680767298569








Bayes Clasification: 
               precision    recall  f1-score   support

           1       0.69      0.24      0.35       155
           2       0.51      0.92      0.65       256
           3       0.52      0.22      0.31       189

    accuracy                           0.52       600
   macro avg       0.57      0.46      0.44       600
weighted avg 

In [34]:
# sample test data predict

msg_test2=["süperonline kullandıkça  mutlu oluyorum "]
y_pred_class = svm.predict(msg_test2)
print(y_pred_class)

['1']


In [35]:
## Cross Validation


svmtfidf = Pipeline([('vect', TfidfVectorizer()), ('svm', LinearSVC())])
scorestfidf = cross_val_score(svmtfidf, tweet_list, label_list, cv=10)  # cross validation k=10

print("SVM Cross Validation (Tf-idf): \n", scorestfidf)
print("SVM Accuracy (Tf-idf): %0.2f (+/- %0.2f)" % (scorestfidf.mean(), scorestfidf.std() * 2), "\n\n")

svmtf = Pipeline([('vect', CountVectorizer()), ('svm', LinearSVC())])
scorestf = cross_val_score(svmtf, tweet_list, label_list, cv=10)  # cross validation k=10

print("SVM Cross Validation (Tf): \n", scorestf)
print("SVM Accuracy (Tf): %0.2f (+/- %0.2f)" % (scorestf.mean(), scorestf.std() * 2), "\n\n")

bayestfidf = Pipeline([('vect', TfidfVectorizer()), ('bayes', MultinomialNB())])
scorestfidf = cross_val_score(bayestfidf, tweet_list, label_list, cv=10)

print("Bayes Cross Validation (Tf-idf): \n", scorestfidf)
print("Bayes Accuracy (Tf-idf): %0.2f (+/- %0.2f)" % (scorestfidf.mean(), scorestfidf.std() * 2), "\n\n")

bayestf = Pipeline([('vect', CountVectorizer()), ('bayes', MultinomialNB())])
scorestf = cross_val_score(bayestf, tweet_list, label_list, cv=10)

print("Bayes Cross Validation (Tf) : \n", scorestf)
print("Bayes Accuracy (Tf): %0.2f (+/- %0.2f)" % (scorestf.mean(), scorestf.std() * 2))


SVM Cross Validation (Tf-idf): 
 [0.6        0.54       0.59666667 0.54333333 0.57666667 0.53666667
 0.6        0.54666667 0.55666667 0.54      ]
SVM Accuracy (Tf-idf): 0.56 (+/- 0.05) 


SVM Cross Validation (Tf): 
 [0.61       0.53666667 0.59333333 0.54       0.56333333 0.56333333
 0.58333333 0.52666667 0.52       0.55666667]
SVM Accuracy (Tf): 0.56 (+/- 0.06) 


Bayes Cross Validation (Tf-idf): 
 [0.59333333 0.55333333 0.56666667 0.55333333 0.56333333 0.52666667
 0.54333333 0.54666667 0.53666667 0.51333333]
Bayes Accuracy (Tf-idf): 0.55 (+/- 0.04) 


Bayes Cross Validation (Tf) : 
 [0.61       0.54666667 0.58333333 0.53666667 0.57333333 0.54
 0.58333333 0.57666667 0.54333333 0.54      ]
Bayes Accuracy (Tf): 0.56 (+/- 0.05)
